In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

In [2]:
#step 1 : Data cleaning
cosm = pd.read_csv('data/cleanser.csv')
cosm.info()

cosm = cosm.loc[pd.notnull(cosm['ingredients'])]
cosm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Label          12 non-null     object 
 1   URL            12 non-null     object 
 2   brand          12 non-null     object 
 3   name           12 non-null     object 
 4   price          12 non-null     object 
 5   rank           12 non-null     float64
 6   skin_type      12 non-null     object 
 7   ingredients    12 non-null     object 
 8   product_image  12 non-null     object 
dtypes: float64(1), object(8)
memory usage: 544.0+ bytes
<class 'pandas.core.frame.DataFrame'>
Int64Index: 12 entries, 0 to 11
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Label          12 non-null     object 
 1   URL            12 non-null     object 
 2   brand          12 non-null     object 
 3   name           12 non-null    

In [3]:
#setting lab
cosm.Label[cosm['Label'] == 'lip-treatments'] = str('Lip Care')

<ipython-input-3-67030a99549d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cosm.Label[cosm['Label'] == 'lip-treatments'] = str('Lip Care')


In [4]:
# name -> duplicated item
df_2 = cosm['name'].drop_duplicates()
cosm = cosm.loc[df_2.index, :].reset_index()

In [5]:
# URL
cosm.drop(['URL', 'index'], axis = 1, inplace = True)

In [6]:
# price
pattern = re.compile(r"(\d+).\d+")
for i in range(len(cosm)):
    cosm['price'][i] = re.findall(pattern, cosm['price'][i])[0]

cosm['price'] = pd.to_numeric(cosm['price'])

<ipython-input-6-22f4ee46374b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cosm['price'][i] = re.findall(pattern, cosm['price'][i])[0]


In [7]:
# rank
cosm['rank'].fillna(0, inplace = True)
cosm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Label          12 non-null     object 
 1   brand          12 non-null     object 
 2   name           12 non-null     object 
 3   price          12 non-null     int64  
 4   rank           12 non-null     float64
 5   skin_type      12 non-null     object 
 6   ingredients    12 non-null     object 
 7   product_image  12 non-null     object 
dtypes: float64(1), int64(1), object(6)
memory usage: 544.0+ bytes


In [8]:
# skin_type
pattern = re.compile(r"([a-zA-Z]+)\\n")
for i in range(len(cosm)):
    cosm['skin_type'][i] = re.findall(pattern, cosm['skin_type'][i])

<ipython-input-8-2878d5c40687>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cosm['skin_type'][i] = re.findall(pattern, cosm['skin_type'][i])


In [9]:
## list column dummies
df_2 = cosm['skin_type'].str.join('|').str.get_dummies()
cosm_2 = cosm.join(df_2).drop('skin_type', axis = 1)

In [10]:
# Split the ingredients with '\r\n\r\n'
a = [t.split('\r\n\r\n') for t in cosm['ingredients']]

# Take onle the part with the real ingredients 
pattern = ['\r\n', '-\w+: ', 'Please', 'No Info', 
           'This product','Visit', 'Features', 'Moisture Wrap™ Technology']

for i in range(len(cosm)):
    num = len(a[i])
    for j in range(num):
        if all(x not in a[i][j] for x in pattern):
            cosm_2['ingredients'][i] = a[i][j]

<ipython-input-10-6f881ff53e69>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cosm_2['ingredients'][i] = a[i][j]


In [11]:
# save the file
cosm_2.to_csv('data/cleanser_p.csv', encoding = 'utf-8-sig', index = False)